# Lab 08: Circular, Array-backed Queue

## Overview

For this assignment you will implement a circular, array-backed queue data structure.

In the following class, which you are to complete, the backing array will be created and populated with `None`s in the `__init__` method, and the `head` and `tail` indexes set to sentinel values (you shouldn't need to modify `__init__`). Enqueueing and Dequeueing items will take place at the tail and head, with `tail` and `head` tracking the position of the most recently enqueued item and that of the next item to dequeue, respectively. To simplify testing, your implementation should make sure that when dequeuing an item its slot in the array is reset to `None`, and when the queue is emptied its `head` and `tail` attributes should be set to `-1`.

Because of the fixed size backing array, the `enqueue` operation is defined to raise a `RuntimeError` when the queue is full — the same exception should be raised when `dequeue` is called on an empty queue.

Finally, the `resize` method will allow the array underlying the queue to be increased in size. It is up to you how to implement this (you can either leave the elements in their current positions, though this may require "unwrapping" elements, or you can simply move all elements towards the front of the array). You may assume that `resize` will only be called with a value greater than the current length of the underlying array.

In [81]:
class Queue:
    def __init__(self, limit=10):
        self.data = [None] * limit
        self.head = -1
        self.tail = -1

    def enqueue(self, val):
        ### BEGIN SOLUTION
        if self.tail == len(self.data)-1:
            raise RuntimeError
        if self.tail == -1:
            self.tail = 0
        else:
            self.tail = (self.tail + 1) % len(self.data)
        self.data[self.tail] = val
        ### END SOLUTION
        
    def dequeue(self):
        ### BEGIN SOLUTION
        if self.empty():
            raise RuntimeError
        self.head = (self.head + 1) % len(self.data)
        ret = self.data[self.head]
        self.data[self.head] = None
        #print(ret)
        
        return ret
        ### END SOLUTION
    
    def resize(self, newsize): #will only *increase* size as stated in overview
        assert(len(self.data) < newsize)
        ### BEGIN SOLUTION
        tmp = len(self.data)-1
        newlst = [None] * (len(self.data)*2) #doubles list size
        for x in range(len(self.data)):
            newlst[x] = self.dequeue()
        self.data = newlst
        self.head = 0
        self.tail = tmp
        ### END SOLUTION
    
    def empty(self):
        ### BEGIN SOLUTION
        for x in self.data:
            if x != None:
                return False
        return True
        ### END SOLUTION
        
    def full(self):
        for x in self.data:
            if x == None:
                return False
        return True
    
    def __bool__(self):
        return not self.empty()
    
    def __str__(self):
        if not(self):
            return ''
        return ', '.join(str(x) for x in self)
    
    def __repr__(self):
        return str(self)
    
    def __iter__(self):
        ### BEGIN SOLUTION
        for x in range(len(self.data)):
            yield self.data[x]
        ### END SOLUTION
        
q = Queue(10)
for i in range(6):
    q.enqueue(i)
print(q)
print("Head: " + str(q.head))
#print(q.tail)
#print()
for i in range(5):
    print(q.dequeue())
    print("Head: " + str(q.head))
    print("Tail: " + str(q.tail))
    print(q)

0, 1, 2, 3, 4, 5, None, None, None, None
Head: -1
0
Head: 0
Tail: 5
None, 1, 2, 3, 4, 5, None, None, None, None
1
Head: 1
Tail: 5
None, None, 2, 3, 4, 5, None, None, None, None
2
Head: 2
Tail: 5
None, None, None, 3, 4, 5, None, None, None, None
3
Head: 3
Tail: 5
None, None, None, None, 4, 5, None, None, None, None
4
Head: 4
Tail: 5
None, None, None, None, None, 5, None, None, None, None


In [82]:
# (5 points)

from unittest import TestCase
tc = TestCase()

q = Queue(5)
tc.assertEqual(q.data, [None] * 5)

for i in range(5):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(5)

for i in range(5):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())

In [83]:
# (5 points)

from unittest import TestCase
tc = TestCase()

q = Queue(10)

for i in range(6):
    q.enqueue(i)
    
tc.assertEqual(q.data.count(None), 4)

for i in range(5):
    q.dequeue()
    
tc.assertFalse(q.empty())
tc.assertEqual(q.data.count(None), 9)
tc.assertEqual(q.head, q.tail)
tc.assertEqual(q.head, 5)

for i in range(9):
    q.enqueue(i)

with tc.assertRaises(RuntimeError):
    q.enqueue(10)

for x, y in zip(q, [5] + list(range(9))):
    tc.assertEqual(x, y)
    
tc.assertEqual(q.dequeue(), 5)
for i in range(9):
    tc.assertEqual(q.dequeue(), i)

tc.assertTrue(q.empty())

AssertionError: 4 != 5

In [69]:
# (5 points)

from unittest import TestCase
tc = TestCase()

q = Queue(5)
for i in range(5):
    q.enqueue(i)
for i in range(4):
    q.dequeue()
for i in range(5, 9):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(10)

q.resize(10)

for x, y in zip(q, range(4, 9)):
    tc.assertEqual(x, y)
    
for i in range(9, 14):
    q.enqueue(i)

for i in range(4, 14):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())
tc.assertEqual(q.head, -1)

RuntimeError: 